In [1]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [2]:
import numpy as np

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
# audio_dataset.ipynb 에서 만든 데이터셋 불러오기

import datasets

label_0_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/label_0_dataset_librosa")
label_1_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/label_1_dataset_librosa")

In [13]:
# model-2 : 추가데이터
angry_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/angry_dataset_hugging")
happy_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/HAPPY_dataset_hugging")
neutral_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/NEUTRAL_dataset_hugging")

ne_data = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/neutral_dataset_hugging_2")
an_data = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/angry_dataset_hugging_2")

In [14]:
an_data

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 12298
})

In [15]:
neutral_dataset

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 1000
})

In [16]:
from datasets import Dataset, concatenate_datasets

minds_local = concatenate_datasets([label_0_dataset, label_1_dataset])
minds_local

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 5501
})

In [17]:
minds_hugging = concatenate_datasets([angry_dataset, happy_dataset, neutral_dataset])
minds_hugging

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 3000
})

In [18]:
label2id = {'angry':'1','neutral':'0'}
id2label = {'1':'angry','0':'neutral'}
num_labels = len(id2label)

In [19]:
def preprocess_function_loacal(examples):
    audio_arrays = [np.array(x["array"]) for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [20]:
minds_local = minds_local.train_test_split(test_size=0.2)

In [21]:
encoded_minds_local = minds_local.map(preprocess_function_loacal, remove_columns="audio", batched=True)

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

In [22]:
encoded_minds_local['train'][0]['input_values'][0]

-0.23423433303833008

In [23]:
type(encoded_minds_local['train'][0]['input_values'][0])

float

In [24]:
def preprocess_function_hugging(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [25]:
minds_hugging = minds_hugging.train_test_split(test_size=0.2)

In [26]:
encoded_minds_hugging = minds_hugging.map(preprocess_function_hugging, remove_columns="audio", batched=True)

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [27]:
encoded_minds_hugging['train'][0]['input_values'][0]

0.0006299205706454813

In [28]:
type(encoded_minds_hugging['train'][0]['input_values'][0])

float

In [29]:
# type(np.array(minds_local['audio'][0]['array']))

In [30]:
encoded_minds_hugging

DatasetDict({
    train: Dataset({
        features: ['intent_class', 'input_values'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['intent_class', 'input_values'],
        num_rows: 600
    })
})

In [31]:
encoded_minds_local

DatasetDict({
    train: Dataset({
        features: ['intent_class', 'input_values'],
        num_rows: 4400
    })
    test: Dataset({
        features: ['intent_class', 'input_values'],
        num_rows: 1101
    })
})

In [32]:
from datasets import DatasetDict, concatenate_datasets

# DatasetDict를 사용하여 두 데이터셋을 하나로 합칩니다.
encoded_minds = DatasetDict({
    "train": concatenate_datasets([encoded_minds_local["train"], encoded_minds_hugging["train"]]),
    "test": concatenate_datasets([encoded_minds_local["test"], encoded_minds_hugging["test"]])
})

In [33]:
type(encoded_minds['train'][0]['input_values'][0])

float

In [34]:
encoded_minds['train'][0]['input_values'][0]

-0.23423433303833008

In [35]:
encoded_minds = encoded_minds.rename_column("intent_class", "label")

In [36]:
encoded_minds

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 6800
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 1701
    })
})

In [37]:
import evaluate

accuracy = evaluate.load("accuracy")

In [38]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [39]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.weight', 'classifier.bias', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
training_args = TrainingArguments(
    output_dir="my_voice_classification_model_4",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    # no_cuda =True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch,Training Loss,Validation Loss,Accuracy
0,0.574700,0.601692,0.656085
2,0.497600,0.511215,0.773075
2,0.388000,0.426779,0.809524
4,0.342900,0.433341,0.801881
4,0.402400,0.427790,0.811287
6,0.380700,0.419561,0.811875
6,0.337300,0.409878,0.811875
8,0.370500,0.398777,0.818930
8,0.290900,0.417184,0.817754
10,0.313200,0.409926,0.824221


TrainOutput(global_step=10600, training_loss=0.2997430364590771, metrics={'train_runtime': 5188.7077, 'train_samples_per_second': 65.527, 'train_steps_per_second': 2.043, 'total_flos': 3.079472191488e+18, 'train_loss': 0.2997430364590771, 'epoch': 49.88})

# 추론